<a href="https://colab.research.google.com/github/muhammadalfateh6/-ALGORYC_Rep/blob/main/LSTM_Sequence_Wise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

data1 = pd.read_csv('Processed Data Aus.csv')

In [ ]:
data1

In [ ]:
data2=data1.reset_index(drop =True)
data2

In [ ]:
data3 = data2.reset_index(drop=True)
data3

In [ ]:
grouped = data3.groupby(["suburb_name_encoded"]).apply(lambda x: x.sort_values(["time"], ascending = True)).reset_index(drop=True)

In [ ]:
grouped1 = grouped.drop(columns=['time','suburb_name'])
grouped1

In [ ]:
grouped2=grouped1.reindex(columns=['user_behavior',
                                   'business_performance',
                                   'value',
                                   'business_reviews',
                                   'business_rating',
                                   'suburb_name_encoded',
                                   'business_category_encoded'])

In [ ]:
grouped2

In [ ]:
grouped3 = grouped2.to_numpy()
grouped3

In [ ]:
grouped3.shape

In [ ]:
grouped3[:, :-1].shape, grouped3[:, -1:].shape

In [ ]:
grouped3

In [ ]:
# Split data into input and output sequences
X, y = [], []
seq_len = 30
# for i in range(len(data) - seq_len):
for i in range(0, len(grouped3) - seq_len, seq_len-1):
    X.append(grouped3[i:i+seq_len, :-1])
    y.append(grouped3[i+seq_len-1, -1:])

X = np.array(X)
y = np.array(y)

# Print shapes of input and output arrays
print('Input shape:', X.shape)
print('Output shape:', y.shape)

In [ ]:
train_size = int(0.7 * len(X))
val_size = int(0.2 * len(X))
test_size = len(X) - train_size - val_size

train_X, val_X, test_X = X[:train_size], X[train_size:train_size+val_size], X[train_size+val_size:]
train_y, val_y, test_y = y[:train_size], y[train_size:train_size+val_size], y[train_size+val_size:]

print("Training data shape:", train_X.shape, train_y.shape)
print("Validation data shape:", val_X.shape, val_y.shape)
print("Test data shape:", test_X.shape, test_y.shape)

In [ ]:
import tensorflow as tf

model1 = tf.keras.Sequential([
    tf.keras.layers.LSTM(128, input_shape=(train_X.shape[1], train_X.shape[2]), return_sequences=True),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.LSTM(64, return_sequences =True),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.LSTM(32, return_sequences =False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(208,activation='relu'),
    tf.keras.layers.Dense(104, activation ='relu'),
    tf.keras.layers.Dense(52, activation ='softmax')
    ])

# callbacks = [EarlyStopping(monitor='accuracy', patience = 15)]
optim = tf.keras.optimizers.Adam(learning_rate=0.001)

model1.compile(optimizer = optim,
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
history = model1.fit(train_X, train_y, 
                     validation_data =(val_X, val_y),
                     batch_size =64,
                     epochs =1000,
                     #callbacks =callbacks,
                     verbose=1) 